In [ ]:
import tensorflow as tf 
import numpy as np
import pandas as pd
import ast
from tensorflow import keras 
import matplotlib.pyplot as pyplot
from keras.layers import Embedding
from keras.utils.np_utils import to_categorical 
from google.colab import drive
from sklearn.preprocessing import OneHotEncoder


In [ ]:
drive.mount('/content/gdrive')
root_path = '/content/gdrive/MyDrive'

Mounted at /content/gdrive


In [ ]:
batch_size = 64  # Batch size for training.
epochs = 64  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 14643  # Number of samples to train on.
max_sequence_length = 30   #length of decoder and encoder sentence length
k = 250  # no of tokens to consider 

In [ ]:
#loading training data 

df = pd.read_csv('/content/gdrive/MyDrive/ASEML/train.csv')
data_train = pd.DataFrame(df,columns=['sourceLineTokens','targetLineTokens'])


#applying ast 
for i in range(len(df)):
    data_train['sourceLineTokens'][i] = ast.literal_eval(df['sourceLineTokens'][i])
for i in range(len(df)):
    data_train['targetLineTokens'][i] = ast.literal_eval(df['targetLineTokens'][i])
print(data_train)













                                  sourceLineTokens                                   targetLineTokens
0                  [if, (, num, %, i, !=, =, 0, )]                       [if, (, num, %, i, ==, 0, )]
1                                               []                                                [}]
2          [d, =, (, b, *, b, ), -, 4, *, a, *, c]         [d, =, (, b, *, b, ), -, 4, *, a, *, c, ;]
3          [printf, (, "Let\\s \", C, " ! ", ), ;]           [printf, (, "Let\\s \", C, \, "!", ), ;]
4                                       [print, f]                                                 []
...                                            ...                                                ...
14638                                          [}]                                             [}, ;]
14639                              [gets, (, ), ;]                     [gets, (, st, [, 100, ], ), ;]
14640        [char, s, [, 100, ], ,, r, [, 100, ]]           [char, s, [, 100, ], 

In [ ]:
#Loading the testing data 


df = pd.read_csv('/content/gdrive/MyDrive/ASEML/valid.csv')
data_test = pd.DataFrame(df,columns=['sourceLineTokens','targetLineTokens'])

print(len(df))
#applying ast 
for i in range(len(df)):
    data_test['sourceLineTokens'][i] = ast.literal_eval(df['sourceLineTokens'][i])
for i in range(len(df)):
    data_test['targetLineTokens'][i] = ast.literal_eval(df['targetLineTokens'][i])

print(data_test)

2584
                                       sourceLineTokens                                   targetLineTokens
0     [if, (, (, factorial, (, x, ), >=, n1, ), &&, ...  [if, (, (, factorial, (, x, ), >=, n1, ), &&, ...
1              [scanf, (, "%d", ,, &, a, [, i, ], ), ;]           [scanf, (, "%d", ,, &, a, [, 0, ], ), ;]
2                                      [a, =, a, /, 10]                                [a, =, a, /, 10, ;]
3                                     [int, k, ,, c, ;]                      [int, k, ,, c, ,, x, ,, y, ;]
4                     [ch, =, rot, (, a, [, s, ], ), ;]                  [ch, =, rot, (, a, [, i, ], ), ;]
...                                                 ...                                                ...
2579  [sum, [, i, ], [, j, ], =, (, 1, +, min, (, su...  [sum, [, i, ], [, j, ], =, (, 1, +, min, (, su...
2580   [scanf, (, "%d", ,, &, rix, [, m, ], [, n, ], )]  [scanf, (, "%d", ,, &, rix, [, m, ], [, n, ], ...
2581  [scanf, (, "%c\n", ,, &, m

In [ ]:
class Vocabulary:
    PAD_token = 0   # Used for padding short sentences
    SOS_token = 1   # Start-of-sentence token
    EOS_token = 2   # End-of-sentence token

    def __init__(self):
        
        self.word2index = {}
        self.word2count = {}
        self.index2word = {}
        self.num_words = 3
        self.num_sentences = 0
        self.longest_sentence = 0

    def add_word(self, word):
        if word not in self.word2index:
            # First entry of word into vocabulary
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            # Word exists; increase word count
            self.word2count[word] += 1
            
    def add_sentence(self, sentence):
        sentence_len = 0
        for word in sentence:
            sentence_len += 1
            self.add_word(word)
        if sentence_len > self.longest_sentence:
            # This is the longest sentence
            self.longest_sentence = sentence_len
        # Count the number of sentences
        self.num_sentences += 1

    def to_word(self, index):
        return self.index2word[index]

    def to_index(self, word):
        return self.word2index[word]

In [ ]:
# Preparing the train data and Dictionary.

v_s = Vocabulary()
v_t = Vocabulary()





dict_source = []
dict_target = []
count = 0



for item in data_train['sourceLineTokens']:
    
     item.insert(0,"SOS")
     item.append("EOS")
    
     n=len(item)
     while(n<max_sequence_length):
       item.append("PAD")
       n+=1



for item in data_train['targetLineTokens']:
    
    item.insert(0,"SOS")
    item.append("EOS")
  
    n=len(item)
    while(n<max_sequence_length):
      item.append("PAD")
      n+=1

for i in range(len(data_train)):
    v_s.add_sentence(data_train['sourceLineTokens'][i])
    v_t.add_sentence(data_train['targetLineTokens'][i])


      

dict_source_count = []
dict_target_count = []

dict_source_count = v_s.word2count
dict_target_count = v_t.word2count 




dict_source_count = sorted(dict_source_count.items(), key=lambda x: x[1],reverse = True)
dict_target_count = sorted(dict_target_count.items(), key=lambda x: x[1],reverse = True)

print(dict_source_count)
print(dict_target_count)
# making dictionary with top k tokens 
for i in range(k):
    dict_source.append(dict_source_count[i][0])
    dict_target.append(dict_target_count[i][0])



dict_source = {key: i for i, key in enumerate(dict_source)}  
dict_target = {key: i for i, key in enumerate(dict_target)}


print(dict_source)    #containing top k tokens
print(type(dict_source)) 



#Adding "OOV" into dictionary 

input_token_index = dict([(item[0], i) for i, item in enumerate(dict_source.items())])  #dictionary for input_token_index
input_token_index["OOV"] = k
target_token_index = dict([(item[0], i) for i, item in enumerate(dict_target.items())])   #dictionary for output_token_index
target_token_index["OOV"] = k

print(input_token_index)
print(target_token_index)



[('PAD', 256021), ('SOS', 29286), ('EOS', 29286), ('(', 16537), (')', 15893), (',', 9209), (';', 8903), ('i', 5628), (']', 5504), ('[', 5490), ('=', 4840), ('printf', 4237), ('a', 3752), ('*', 3152), ('+', 2674), ('1', 2569), ('0', 2446), ('j', 2391), ('n', 2376), ('-', 2136), ('if', 2105), ('int', 2048), ('{', 2001), ('b', 1687), ('==', 1679), ('&', 1468), ('c', 1396), ('&&', 1299), ('"%d"', 1245), ('}', 1213), ('++', 1148), ('<', 1078), ('for', 1069), ('k', 982), ('scanf', 972), ('.', 943), ('2', 884), ('<=', 835), ('/', 817), ('x', 810), ('str', 743), ('||', 735), ('m', 717), ('!=', 713), ('%', 677), ('\\', 663), ('r', 662), ('else', 595), ('>', 585), ('s', 551), ('y', 498), ('d', 467), ('p', 447), ('>=', 438), ('arr', 414), ('N', 412), ('"%c"', 408), ('char', 408), ('return', 387), ('3', 377), ('count', 363), ('float', 326), ('l', 314), ('4', 309), ('x1', 309), ('100', 306), ('sum', 303), ('10', 287), ('y1', 281), ('while', 276), ('A', 273), ('->', 266), ('ch', 251), ('line', 239),

In [ ]:
#Check if PAD,OOV,SOS,EOS done Succesfully on Train Data
print(data_train['sourceLineTokens'])
print(data_train['targetLineTokens'])

In [ ]:
# Preparing the test data.

for item in data_test['sourceLineTokens']:
    
     item.insert(0,"SOS")
     item.append("EOS")
    
     n=len(item)
     while(n<max_sequence_length):
       item.append("PAD")
       n+=1



for item in data_test['targetLineTokens']:
    
    item.insert(0,"SOS")
    item.append("EOS")
  
    n=len(item)
    while(n<max_sequence_length):
      item.append("PAD")
      n+=1

      


In [ ]:
#Check if PAD,OOV,SOS,EOS done Succesfully on Test Data
print(data_test['sourceLineTokens'])
print(data_test['targetLineTokens'])

0       [SOS, SOS, if, (, (, factorial, (, x, ), >=, n...
1       [SOS, SOS, scanf, (, "%d", ,, &, a, [, i, ], )...
2       [SOS, SOS, a, =, a, /, 10, EOS, PAD, PAD, PAD,...
3       [SOS, SOS, int, k, ,, c, ;, EOS, PAD, PAD, PAD...
4       [SOS, SOS, ch, =, rot, (, a, [, s, ], ), ;, EO...
                              ...                        
2579    [SOS, SOS, sum, [, i, ], [, j, ], =, (, 1, +, ...
2580    [SOS, SOS, scanf, (, "%d", ,, &, rix, [, m, ],...
2581    [SOS, SOS, scanf, (, "%c\n", ,, &, matrix, [, ...
2582    [SOS, SOS, scanf, (, "%f %f %f %f %f %f ", ,, ...
2583    [SOS, SOS, g, =, 1, ;, }, }, EOS, PAD, PAD, PA...
Name: sourceLineTokens, Length: 2584, dtype: object
0       [SOS, SOS, if, (, (, factorial, (, x, ), >=, n...
1       [SOS, SOS, scanf, (, "%d", ,, &, a, [, 0, ], )...
2       [SOS, SOS, a, =, a, /, 10, ;, EOS, PAD, PAD, P...
3       [SOS, SOS, int, k, ,, c, ,, x, ,, y, ;, EOS, P...
4       [SOS, SOS, ch, =, rot, (, a, [, i, ], ), ;, EO...
                    

In [ ]:
#encoding for train 


source_data_onehot = []
target_data_onehot = []


for src in data_train['sourceLineTokens']:
  
  
  lst = []
  for item2 in src:
    if input_token_index.get(item2) == None:
      lst.append(k)
    else:
      lst.append(input_token_index[item2])

  source_data_onehot.append(lst)    


for tgt in data_train['targetLineTokens']:
  
 
  lst = []
  for item2 in tgt:
    if target_token_index.get(item2) == None:
      lst.append(k)
    else:
      lst.append(target_token_index[item2])
  
  target_data_onehot.append(lst)  



#padding and truncating 

source_data_onehot = keras.preprocessing.sequence.pad_sequences(source_data_onehot,maxlen = max_sequence_length,padding= 'post',truncating= 'post')  #to be converted into OneHot
target_data_onehot = keras.preprocessing.sequence.pad_sequences(target_data_onehot,maxlen = max_sequence_length,padding= 'post',truncating= 'post')













In [ ]:
print(source_data_onehot[0])
print(target_data_onehot[0])


[ 3 20  1 83 44  7 43 10 16  2  4  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0]
[ 3 19  1 76 51  7 22 16  2  4  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0]


In [ ]:
#encoding for test 


test_source_data_onehot = []
test_target_data_onehot = []

for src in data_test['sourceLineTokens']:
  
  lst = []
  for item2 in src:
    if input_token_index.get(item2) == None:
      lst.append(k)
    else:
      lst.append(input_token_index[item2])

  test_source_data_onehot.append(lst)    


for tgt in data_test['targetLineTokens']:
  
  
  lst = []
  for item2 in tgt:
    if target_token_index.get(item2) == None:
      lst.append(k)
    else:
      lst.append(target_token_index[item2])
  
  test_target_data_onehot.append(lst)

#padding and truncating  test data 

test_source_data_onehot = keras.preprocessing.sequence.pad_sequences(test_source_data_onehot,maxlen = max_sequence_length,padding= 'post',truncating= 'post')
test_target_data_onehot = keras.preprocessing.sequence.pad_sequences(test_target_data_onehot,maxlen = max_sequence_length,padding= 'post',truncating= 'post')



In [ ]:
print(test_source_data_onehot[0])
print(test_target_data_onehot[0])

[  3  20   1   1 250   1  39   2  53  93   2  27   1 250   1  39   2  37
 113   2   2  22   4   0   0   0   0   0   0   0]
[  3  19   1   1 250   1  36   2  50  89   2  27   1 250   1  36   2  35
 108   2   2  21   4   0   0   0   0   0   0   0]


In [ ]:
# k = 250
# k = k + 1

In [ ]:
#One Hot Encoding for Train
#Encoder Input Data

encoder_input_data = np.zeros(
    (len(data_train), max_sequence_length, k+1), dtype="float32"
)

for i in range(len(source_data_onehot)):
  for j in range(len(source_data_onehot[i])):
    
    kj = source_data_onehot[i][j]
    encoder_input_data[i][j][kj] = 1.0


# Decoder_input data

decoder_input_data = np.zeros(
    (len(data_train), max_sequence_length, k+1), dtype="float32"
)

for i in range(len(target_data_onehot)):
  for j in range(len(target_data_onehot[i])):
    kj = target_data_onehot[i][j]
    decoder_input_data[i][j][kj] = 1.0
    
#Decoder Target Data 

decoder_target_data = np.zeros(
    (len(data_train), max_sequence_length, k+1), dtype="float32"
)

for i in range(len(data_train)):
  for j in range(1, max_sequence_length):
    decoder_target_data[i][j-1] = decoder_input_data[i][j]

decoder_target_data[i][-1][0] = 1.0  

In [ ]:
#One Hot Encoding for Test
#Encoder Input Data




test_encoder_input_data = np.zeros(
    (len(data_test), max_sequence_length, k+1), dtype="float32"
)

for i in range(len(test_source_data_onehot)):
  for j in range(len(test_source_data_onehot[i])):
    kj = test_source_data_onehot[i][j]
    test_encoder_input_data[i][j][kj] = 1.0




In [ ]:

# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, k+1))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, k+1))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(k+1, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split = 0.2,
)
# Save model
model.save("/content/gdrive/MyDrive/ASEML/model")

Epoch 1/64
184/184 [==============================] - 6s 19ms/step - loss: 1.1459 - accuracy: 0.8133 - val_loss: 0.5832 - val_accuracy: 0.8682
Epoch 2/64
184/184 [==============================] - 3s 14ms/step - loss: 0.3349 - accuracy: 0.9151 - val_loss: 0.5402 - val_accuracy: 0.8876
Epoch 3/64
184/184 [==============================] - 3s 14ms/step - loss: 0.2433 - accuracy: 0.9366 - val_loss: 0.4341 - val_accuracy: 0.9088
Epoch 4/64
184/184 [==============================] - 3s 14ms/step - loss: 0.1982 - accuracy: 0.9480 - val_loss: 0.4162 - val_accuracy: 0.9087
Epoch 5/64
184/184 [==============================] - 3s 14ms/step - loss: 0.1705 - accuracy: 0.9553 - val_loss: 0.3769 - val_accuracy: 0.9162
Epoch 6/64
184/184 [==============================] - 3s 14ms/step - loss: 0.1489 - accuracy: 0.9609 - val_loss: 0.4338 - val_accuracy: 0.9098
Epoch 7/64
184/184 [==============================] - 3s 15ms/step - loss: 0.1341 - accuracy: 0.9649 - val_loss: 0.3989 - val_accuracy: 0.9152

INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/ASEML/model/assets


INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/ASEML/model/assets


In [ ]:
# Define sampling models
# Restore the model and construct the encoder and decoder.
model = keras.models.load_model("/content/gdrive/MyDrive/ASEML/model")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,), name="input_3")
decoder_state_input_c = keras.Input(shape=(latent_dim,), name="input_4t")
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.

reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, k+1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["SOS"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = []
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence.append(sampled_char)

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "EOS" or len(decoded_sentence) > max_sequence_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, k+1))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence




In [ ]:
print(reverse_target_char_index)

{0: 'PAD', 1: '(', 2: ')', 3: 'SOS', 4: 'EOS', 5: ';', 6: ',', 7: 'i', 8: '[', 9: ']', 10: 'printf', 11: '=', 12: 'a', 13: '*', 14: '+', 15: '1', 16: '0', 17: 'j', 18: '-', 19: 'if', 20: 'n', 21: '{', 22: '==', 23: 'int', 24: 'b', 25: '&', 26: 'c', 27: '&&', 28: '}', 29: '"%d"', 30: '++', 31: 'for', 32: 'k', 33: '<', 34: 'scanf', 35: '<=', 36: 'x', 37: '2', 38: '.', 39: '!=', 40: 'r', 41: '||', 42: 'm', 43: 'str', 44: '/', 45: '>', 46: 's', 47: 'y', 48: 'else', 49: '\\', 50: '>=', 51: '%', 52: 'p', 53: 'N', 54: '"%c"', 55: 'return', 56: 'arr', 57: 'd', 58: 'count', 59: 'char', 60: '3', 61: '100', 62: '->', 63: 'x1', 64: 'l', 65: 'y1', 66: '10', 67: '4', 68: 'sum', 69: 'float', 70: 'while', 71: 'ch', 72: 'x2', 73: 'a1', 74: 'y2', 75: 'A', 76: 'num', 77: 't', 78: 'a2', 79: '"%s"', 80: 's1', 81: "'a'", 82: 'line', 83: 'area', 84: 'b1', 85: 'b2', 86: 'new', 87: "'\\0'", 88: '"%f"', 89: 'n1', 90: 'h', 91: "'A'", 92: 'pi', 93: 'z', 94: '"%d\\n"', 95: 'mat', 96: 'struct', 97: 'sizeof', 98: '"

In [ ]:
input_texts = data_test['sourceLineTokens']
cnt = 0
output = []

for seq_index in range(len(data_test)):
  input_seq = test_encoder_input_data[seq_index : seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)

  target_token = data_test["targetLineTokens"][seq_index]
  
  target_check = []
  for i in range(len(target_token)):
    if target_token[i] not in input_token_index:
      target_check.append("OOV")
    elif target_token[i] not in ['SOS', 'EOS', 'PAD']:
      target_check.append(target_token[i])
  output_check = []
  for i in range(len(decoded_sentence)):
    if decoded_sentence[i] not in ['SOS', 'EOS', 'PAD']:
      output_check.append(decoded_sentence[i])

  chk = 1
  for i in range(len(target_check)):
    if i == max_sequence_length or i == len(output_check):
      chk = 0
      break
    if output_check[i] != target_check[i]:
      chk = 0
      break
  
  output.append(output_check)
  
  if chk:
    cnt += 1
    print(seq_index, " = ", cnt, end = ", ")
  
print(cnt)

2  =  1, 7  =  2, 11  =  3, 14  =  4, 15  =  5, 19  =  6, 20  =  7, 21  =  8, 23  =  9, 29  =  10, 30  =  11, 33  =  12, 34  =  13, 35  =  14, 38  =  15, 47  =  16, 48  =  17, 49  =  18, 51  =  19, 52  =  20, 57  =  21, 60  =  22, 61  =  23, 62  =  24, 63  =  25, 67  =  26, 69  =  27, 70  =  28, 71  =  29, 73  =  30, 76  =  31, 79  =  32, 80  =  33, 81  =  34, 82  =  35, 83  =  36, 87  =  37, 98  =  38, 104  =  39, 105  =  40, 107  =  41, 108  =  42, 110  =  43, 115  =  44, 116  =  45, 119  =  46, 120  =  47, 122  =  48, 123  =  49, 127  =  50, 130  =  51, 131  =  52, 136  =  53, 137  =  54, 142  =  55, 145  =  56, 148  =  57, 150  =  58, 159  =  59, 163  =  60, 164  =  61, 165  =  62, 167  =  63, 172  =  64, 177  =  65, 181  =  66, 182  =  67, 184  =  68, 185  =  69, 186  =  70, 187  =  71, 189  =  72, 192  =  73, 195  =  74, 197  =  75, 201  =  76, 202  =  77, 205  =  78, 207  =  79, 210  =  80, 216  =  81, 219  =  82, 222  =  83, 223  =  84, 225  =  85, 226  =  86, 237  =  87, 240  

In [ ]:
accuracy = cnt/len(data_test)
print(accuracy)
data_test = pd.read_csv('/content/gdrive/MyDrive/ASEML/valid.csv')
data_test["fixedTokens"] = output
data_test.to_csv("/content/gdrive/MyDrive/ASEML/predicted_output3.csv")

0.3459752321981424


# New Section

# New Section

In [ ]:
# print(output)